In [13]:
import warnings
warnings.filterwarnings('ignore')

In [14]:
from dataloader.dataloader import training_dataset, testing_dataset
import torch
from torch.utils.data import DataLoader
from architecture.pipe import get_model
import yaml
from tqdm import tqdm
import os

In [15]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [31]:
training_data = DataLoader(training_dataset, batch_size=32, shuffle=True, num_workers=4, pin_memory=True)
testing_data = DataLoader(testing_dataset, batch_size=32, shuffle=True, num_workers=4, pin_memory=True)

In [37]:
vision_model = get_model()

In [38]:
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(vision_model.parameters(), lr=1e-4)

In [39]:
state = {'loss': [], 'epochs': [], 'test_loss': []}
def train_loop(dataloader, testing_data, model, loss_fn, optimizer, epochs=10, save_path='./saves'):
    global state
    if not os.path.exists(save_path):
        os.mkdir(save_path)

    all_exps = [int(elm.replace('exp_', '')) if elm != '.ipynb_checkpoints' else -1 for elm in os.listdir(save_path)]

    current_num = max(all_exps) if (len(all_exps) > 0) else 0
    save_path = save_path + '/exp_' + str(current_num + 1)
    os.mkdir(save_path)

    model.to(device)

    for epoch in range(epochs):
        print(f"Epoch {epoch + 1}/{epochs}")
        model.train()
        loop = tqdm(dataloader, total=len(dataloader), leave=True)
        total_loss = 0

        for batch_idx, (data, targets) in enumerate(loop):
            data = data.to(device)
            targets = targets.to(device)

            # Forward pass
            scores = model(data)
            loss = loss_fn(scores, targets)

            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

            # Update progress bar
            loop.set_postfix(loss=loss.item())

        model.eval()
        testing_loss = 0
        for test in tqdm(testing_data):
            X, y = test
            
            X, y = X.to(device), y.to(device)
            
            with torch.no_grad():                
                score = model(X)
                testing_loss =+ loss_fn(score, y)

        model.train()

        print(f"Epoch {epoch + 1} average loss: {total_loss / len(dataloader)} with testing loss of {testing_loss / len(testing_data)}")

        state['loss'].append(total_loss / len(dataloader))
        state['epochs'].append(epoch + 1)
        state['test_loss'].append((testing_loss / len(testing_data)).cpu())

        # Save the models after each epoch
        torch.save({
            'epoch': epoch + 1,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': total_loss / len(dataloader),
            'testing_loss': (testing_loss / len(testing_data)).cpu()
        }, f"{save_path}/model_epoch_{epoch + 1}.pth")

    print("Training complete!")

In [40]:
train_loop(training_data, testing_data, vision_model, loss_fn, optimizer, epochs=10)

Epoch 1/10


100%|██████████| 19/19 [00:22<00:00,  1.19s/it]


Epoch 1 average loss: 2.2331746792055895 with testing loss of 0.13195718824863434
Epoch 2/10


100%|██████████| 19/19 [00:23<00:00,  1.22s/it]


Epoch 2 average loss: 2.1545107843949625 with testing loss of 0.10787514597177505
Epoch 3/10


...

Epoch 8 average loss: 2.1038240956276963 with testing loss of 0.10429888218641281
Epoch 9/10


100%|██████████| 19/19 [00:22<00:00,  1.18s/it]


Epoch 9 average loss: 2.1003783093285313 with testing loss of 0.141468808054924
Epoch 10/10


100%|██████████| 19/19 [00:23<00:00,  1.24s/it]


Epoch 10 average loss: 2.1003942587940965 with testing loss of 0.10800635069608688
Training complete!


In [ ]:
state.keys()